# ELT

In [4]:
import os 
import pandas as pd
import boto3
import yaml
import awswrangler as wr


# Extract and Load


## Creacion de Tabla Inflacion por Athena

In [ ]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`inflacion` (
    `date` DATE,
    `inflacion` DOUBLE
    ) COMMENT "Data de Inflacion de mexico."
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-mau/econ/raw/inflacion/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [9]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False,
)

""


## Creacion de Tabla Tasa de Interes por Athena

In [10]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tasa_de_interes` (
    `date` DATE,
    `tasa_de_interes` DOUBLE
    ) COMMENT "Data de tasa de interes de mexico."
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-mau/econ/raw/tasa/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [11]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False,
)

""


## Creacion de Tabla tipo de cambio por Athena

In [12]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tipo_de_cambio` (
    `date` DATE,
    `tipo_de_cambio` DOUBLE
    ) COMMENT "Data de tipo de cambio por dolar en mexico."
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-mau/econ/raw/tipo_cambio/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [13]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False,
)

""


## Transform

In [16]:

query = '''
CREATE TABLE IF NOT EXISTS econ.data_mensual AS 
SELECT 
    DATE_TRUNC('month', inflacion.date) AS date, 
    AVG(inflacion.inflacion) AS inflacion, 
    AVG(tasa_de_interes.tasa_de_interes) AS tasa_de_interes, 
    AVG(tipo_de_cambio.tipo_de_cambio) AS tipo_de_cambio
FROM inflacion
INNER JOIN tasa_de_interes ON inflacion.date = tasa_de_interes.date
INNER JOIN tipo_de_cambio ON tasa_de_interes.date = tipo_de_cambio.date
GROUP BY DATE_TRUNC('month', inflacion.date)
ORDER BY date;
'''

In [17]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False)

""
